In [1]:
import mne
import os
from tqdm import tqdm
import matlab.engine
import numpy as np
mne.set_log_level(40)
eng = matlab.engine.start_matlab()

In [2]:
for fname in tqdm(os.listdir('data')):
    raw = mne.io.read_raw_gdf('data/'+fname, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    map = {
        'EEG-Fz': 'Fz',
        'EEG-0': 'FC3',
        'EEG-1': 'FC1',
        'EEG-2': 'FCz',
        'EEG-3': 'FC2',
        'EEG-4': 'FC4',
        'EEG-5': 'C5',
        'EEG-C3': 'C3',
        'EEG-6': 'C1',
        'EEG-Cz': 'Cz',
        'EEG-7': 'C2',
        'EEG-C4': 'C4',
        'EEG-8': 'C6',
        'EEG-9': 'CP3',
        'EEG-10': 'CP1',
        'EEG-11': 'CPz',
        'EEG-12': 'CP2',
        'EEG-13': 'CP4',
        'EEG-14': 'P1',
        'EEG-Pz': 'Pz',
        'EEG-15': 'P2',
        'EEG-16': 'Oz',
        'EOG-left': 'EoG1',
        'EOG-central': 'EoG2',
        'EOG-right': 'EoG3'
    }
    mne.rename_channels(raw.info, mapping=map)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)
    
    ica = mne.preprocessing.ICA(
        n_components=10,
        max_iter='auto',
        random_state=10
    )
    ica.fit(raw)
    eog_indices, ecg_scores = ica.find_bads_eog(raw, threshold='auto')
    ica.exclude = eog_indices
    raw = ica.apply(raw.copy(), exclude=ica.exclude)
    
    this_participant = eng.load('mat/'+fname.replace('.gdf', '.mat'))
    for data in this_participant['data']:
        if len(np.asarray(data['trial'])) != 0:
                trial = np.asarray(data['trial']).squeeze()
                classes = np.asarray(data['classes'])
                y = np.asarray(data['y'])
                description = [classes[int(x-1)] for x in y]
                annotations = mne.Annotations(onset=trial/raw.info['sfreq'], duration=0, description=description)
                raw.set_annotations(annotations)
    
    raw.save('epochs/'+fname.replace('.gdf', '.fif'), picks='eeg')

100%|██████████| 18/18 [04:41<00:00, 15.61s/it]
